In [179]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import os

In [180]:
url_base = "https://www.gasolinamx.com/estado/ciudad-de-mexico-cdmx/"
tipo_combustible = ["Magna", "Premium", "Diesel"]
delegaciones_cdmx = ['alvaro-obregon', 'azcapotzalco', 'benito-juarez', 'coyoacan', 'cuajimalpa-de-morelos', 'cuauhtemoc',
                     'gustavo-a-madero', 'iztacalco', 'iztapalapa', 'la-magdalena-contreras', 'miguel-hidalgo', 'milpa-alta', 
                     'tlahuac', 'tlalpan', 'venustiano-carranza', 'xochimilco']
fecha = datetime.datetime.now().strftime("%d-%m-%Y")
data = []

In [181]:
def obtener_contneido_pagina(url):
    pagina = requests.get(url)
    return pagina.content

In [182]:
def genera_soup(html):
    return BeautifulSoup(html, "html.parser") 

In [183]:
def procesar_pagina(soup, fecha, delegacion):

    tr_magna = soup.find("tr", class_ = "bg-green")
    precio_magna = tr_magna.text[6:].strip()

    tr_premium = soup.find("tr", class_ = "bg-red")
    precio_premium = tr_premium.text[9:].strip()

    tr_diesel = soup.find("tr", class_ = "bg-black")
    precio_diesel = tr_diesel.text[7:].strip()

    precios = [precio_magna, precio_premium, precio_diesel]
    
    for i in range(len(precios)):
        data.append({"Fecha" : fecha,
                     "Delegacion" : delegacion,
                     "Combustible" : tipo_combustible[i],
                     "Precio" : precios[i]})
      
    return data

In [184]:
def manejar_paginacion(url_base):
    for delegacion in delegaciones_cdmx:
        url = url_base + delegacion
        html = obtener_contneido_pagina(url)
        soup = genera_soup(html)
        procesar_pagina(soup, fecha, delegacion)
    return pd.DataFrame(data)

In [178]:
df_hoy = manejar_paginacion(url_base)
def actualiza_base(df_hoy):

    # Crear una lista con los documentos en la carpeta
    documentos = os.listdir("../csv_gasolina")
    # Al ultimo documento de la lista, extraer n_update y fecha
    n_update = int(documentos[-2].split()[0]) 
    new_n_update = str(n_update + 1) 
    fecha_nombre_anterior = documentos[-2].split()[1][:-4]
    # Sumarle un dia a la fecha del nombre anterior
    from datetime import datetime, timedelta
    fecha_nombre_nuevo = datetime.strptime(fecha_nombre_anterior, "%d-%m-%Y") + timedelta(days=1)
    # Convertir a objeto str
    fecha_nombre_nuevo = fecha_nombre_nuevo.strftime("%d-%m-%Y")

    df_base = pd.read_csv("../csv_gasolina/" + str(n_update) + " " + fecha_nombre_anterior + ".csv") # Actualizar el nombre que ya no es la base
    df_nuevo = pd.concat([df_base, df_hoy], ignore_index=True)
    datos_utiles = ["Fecha", "Delegacion", "Combustible", "Precio"]
    df_nuevo = df_nuevo[datos_utiles]
    df_nuevo.to_csv("../csv_gasolina/"+ new_n_update + " " + fecha_nombre_nuevo + ".csv")